# Transformer Implementation

Following [The Annotated Transformer](https://nlp.seas.harvard.edu/annotated-transformer/)

## Prelims

In [4]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR

# import pandas as pd
import polars as pl  # Polars because we're cool like that
import altair as alt

from torch.utils.data import DataLoader

import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

# # Can't import torchtext with ROCm
# from torchtext.data.functional import to_map_style_dataset
# from torchtext.vocab import build_vocab_from_iterator
# import torchtext.datasets as datasets

In [5]:
# Set to False to skip notebook execution (e.g. for debugging)
RUN_EXAMPLES = True

In [ ]:
# Convenience helper functions
def is_interactive_noteboot():
    return __name__ == "__main__"